# ETL for recreating the paper "Forecasting the direction of the Fed's monetary policy decisions using random forest"

## Majority of variables able to grab from yfinance and FRED

In [15]:
import pandas as pd
import numpy as np
from fredapi import Fred
import yfinance as yf
from datetime import datetime

# need to have a FRED API key
fred = Fred(api_key='add your key here')

VARIABLES = {
    'CPIAUCSL': {'freq': 'monthly', 'transformation': 'QoQ'},
    'CPILFESL': {'freq': 'monthly', 'transformation': 'QoQ'},
    'PPIACO': {'freq': 'monthly', 'transformation': 'QoQ'},
    'CL=F': {'source': 'yfinance', 'ticker': 'CL=F', 'freq': 'daily', 'transformation': 'QoQ'},

    # Group 2: Real Activity
    'GDP': {'freq': 'quarterly', 'transformation': 'QoQ'},
    'GDPC1': {'freq': 'quarterly', 'transformation': 'QoQ'},
    'INDPRO': {'freq': 'monthly', 'transformation': 'QoQ'},
    'UNRATE': {'freq': 'monthly', 'transformation': 'QoQ'},
    'PAYEMS': {'freq': 'monthly', 'transformation': 'QoQ'},
    'MANEMP': {'freq': 'monthly', 'transformation': 'QoQ'},
    'ICSA': {'freq': 'weekly', 'transformation': 'QoQ'},
    'IC4WSA': {'freq': 'weekly', 'transformation': 'QoQ'},
    'RRSFS': {'freq': 'monthly', 'transformation': 'QoQ'},

    # Group 3: Financial Variables
    # spy 
    '^GSPC': {'source': 'yfinance', 'ticker': '^GSPC', 'freq': 'daily', 'transformation': 'QoQ'},
    # vix
    '^VIX': {'source': 'yfinance', 'ticker': '^VIX', 'freq': 'daily', 'transformation': 'QoQ'},
    'DX-Y.NYB': {'source': 'yfinance', 'ticker': 'DX-Y.NYB', 'freq': 'daily', 'transformation': 'QoQ'},

    # Group 4: Monetary Variables
    'M2SL': {'freq': 'weekly', 'transformation': 'QoQ'},
    'TB6SMFFM': {'freq': 'monthly', 'transformation': 'level'},
    'FEDFUNDS': {'freq': 'monthly', 'transformation': 'level'},
    'T10Y2Y': {'freq': 'daily', 'transformation': 'level'},

    # misc: Treasury Rates to compute spreads
    'GS3M': {'freq': 'daily', 'transformation': 'level'},
    'GS2': {'freq': 'daily', 'transformation': 'level'},
    'GS5': {'freq': 'daily', 'transformation': 'level'},
}

def fetch_data(var_name, config):
    if config.get('source') == 'yfinance':
        df = yf.download(config['ticker'], start='1994-01-01')
        print(f"Grabbed {var_name} from yfinance")
        data = df['Close'].copy()
        print(data.head())
        data.name = var_name
    else:
        series_id = config.get('series_id', var_name)
        data = fred.get_series(series_id)
        data.name = var_name
    return data

data_all = pd.DataFrame()

for var_name, config in VARIABLES.items():
    print(f"Fetching {var_name}...")
    try:
        data = fetch_data(var_name, config)
        if not isinstance(data.index, pd.DatetimeIndex):
            data.index = pd.to_datetime(data.index)
        data_all = pd.concat([data_all, data], axis=1)
    except Exception as e:
        print(f"Failed to fetch {var_name}: {str(e)}")

data_all.index = pd.to_datetime(data_all.index)

if all(col in data_all.columns for col in ['GS2', 'GS3M']):
    data_all['T2Y3M'] = data_all['GS2'] - data_all['GS3M']
if all(col in data_all.columns for col in ['GS5', 'GS2']):
    data_all['T5Y2Y'] = data_all['GS5'] - data_all['GS2']

if 'FEDFUNDS' in data_all.columns:
    data_all['prev1_decision'] = data_all['FEDFUNDS'].shift(1)
    data_all['prev2_decision'] = data_all['FEDFUNDS'].shift(2)

data_all.drop(columns=['GS2', 'GS3M', 'GS5', 'FEDFUNDS'], inplace=True)

data_q = data_all.resample('Q').last()
print(data_q.columns)
for var_name, config in VARIABLES.items():
    try:
        if var_name in data_q.columns and config['transformation'] == 'QoQ':
            print(f"qoq {var_name}...")
            data_q[f'{var_name}_QoQ'] = data_q[var_name].pct_change(1) * 100
    except Exception as e:
        print(f"Failed to transform {var_name}: {str(e)}")

cols_to_keep = []
for var_name, config in VARIABLES.items():
    if var_name in data_q.columns:
        cols_to_keep.append(f'{var_name}_QoQ' if config['transformation'] == 'QoQ' else var_name)

# add columns we have to manually calculate
cols_to_keep.extend(['T2Y3M', 'T5Y2Y', 'prev1_decision', 'prev2_decision'])

data_final = data_q[cols_to_keep]

Fetching CPIAUCSL...
Fetching CPILFESL...
Fetching PPIACO...


[*********************100%***********************]  1 of 1 completed

Fetching CL=F...
Grabbed CL=F from yfinance
Ticker           CL=F
Date                 
2000-08-23  32.049999
2000-08-24  31.629999
2000-08-25  32.049999
2000-08-28  32.869999
2000-08-29  32.720001


Fetching GDP...
Fetching GDPC1...
Fetching INDPRO...
Fetching UNRATE...
Fetching PAYEMS...
Fetching MANEMP...
Fetching ICSA...
Fetching IC4WSA...
Fetching RRSFS...
Fetching ^GSPC...


[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


Grabbed ^VIX from yfinance
Ticker       ^VIX
Date             
1994-01-03  12.57
1994-01-04  11.91
1994-01-05  10.94
1994-01-06  11.27
1994-01-07  10.96
Fetching DX-Y.NYB...


[*********************100%***********************]  1 of 1 completed


Grabbed DX-Y.NYB from yfinance
Ticker       DX-Y.NYB
Date                 
1994-01-03  96.970001
1994-01-04  96.550003
1994-01-05  96.580002
1994-01-06  96.820000
1994-01-07  96.080002
Fetching M2SL...
Fetching TB6SMFFM...
Fetching FEDFUNDS...
Fetching T10Y2Y...
Fetching GS3M...
Fetching GS2...
Fetching GS5...
Index(['CPIAUCSL', 'CPILFESL', 'PPIACO', 'CL=F', 'GDP', 'GDPC1', 'INDPRO',
       'UNRATE', 'PAYEMS', 'MANEMP', 'ICSA', 'IC4WSA', 'RRSFS', '^GSPC',
       '^VIX', 'DX-Y.NYB', 'M2SL', 'TB6SMFFM', 'T10Y2Y', 'T2Y3M', 'T5Y2Y',
       'prev1_decision', 'prev2_decision'],
      dtype='object')
qoq CPIAUCSL...
qoq CPILFESL...
qoq PPIACO...
qoq CL=F...
qoq GDP...
qoq GDPC1...
qoq INDPRO...
qoq UNRATE...
qoq PAYEMS...
qoq MANEMP...
qoq ICSA...
qoq IC4WSA...
qoq RRSFS...
qoq ^GSPC...
qoq ^VIX...
qoq DX-Y.NYB...
qoq M2SL...


/tmp/ipykernel_66875/3636239974.py:90: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  data_q[f'{var_name}_QoQ'] = data_q[var_name].pct_change(1) * 100
/tmp/ipykernel_66875/3636239974.py:90: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  data_q[f'{var_name}_QoQ'] = data_q[var_name].pct_change(1) * 100
/tmp/ipykernel_66875/3636239974.py:90: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  data_q[f'{var_name

In [16]:
# filter to keep only data from 1994 onward
data_final = data_final[data_final.index >= '1994-01-01']
# data_final.drop(columns=['GS2', 'GS3M', 'GS5'], inplace=True)

# verify the new date range
print(f"New date range: {data_final.index.min()} to {data_final.index.max()}")

New date range: 1994-03-31 00:00:00 to 2025-06-30 00:00:00


In [17]:
nan_counts = data_final.isna().sum(axis=1)

# Get rows with most NaN values (sorted descending)
rows_with_most_nans = nan_counts.sort_values(ascending=False)

# Display top 5 rows with most NaN values
print("Rows with most NaN values:")
print(rows_with_most_nans.head())

# Optionally display the actual data for these rows
print("\nData for these rows:")
print(data_final.loc[rows_with_most_nans.head().index])

Rows with most NaN values:
2025-06-30    5
1994-03-31    4
1995-06-30    1
1994-12-31    1
2000-09-30    1
dtype: int64

Data for these rows:
            CPIAUCSL_QoQ  CPILFESL_QoQ  PPIACO_QoQ   CL=F_QoQ   GDP_QoQ  \
2025-06-30      0.000000      0.000000    0.000000 -12.856746  0.000000   
1994-03-31      0.546822      0.648088    0.927487        NaN  1.453063   
1995-06-30      0.793651      0.750469    1.129944        NaN  0.780454   
1994-12-31      0.535834      0.507937    0.743802        NaN  1.694335   
2000-09-30      0.813008      0.662617    0.672646        NaN  0.687421   

            GDPC1_QoQ  INDPRO_QoQ  UNRATE_QoQ  PAYEMS_QoQ  MANEMP_QoQ  ...  \
2025-06-30   0.000000    0.000000    0.000000    0.000000    0.000000  ...   
1994-03-31   0.970363    1.378298    0.000000    0.819949    0.487660  ...   
1995-06-30   0.298323    0.635666    3.703704    0.327608   -0.092684  ...   
1994-12-31   1.145570    2.498505   -6.779661    0.803492    0.607690  ...   
2000-09-30   0.10

In [18]:
# the fed funds and 6-month Treasury Bill minus federal funds rate is missing for 6-30-2025 but we an drop this row as its not needed 
print(data_final.columns)
data_final.tail()

Index(['CPIAUCSL_QoQ', 'CPILFESL_QoQ', 'PPIACO_QoQ', 'CL=F_QoQ', 'GDP_QoQ',
       'GDPC1_QoQ', 'INDPRO_QoQ', 'UNRATE_QoQ', 'PAYEMS_QoQ', 'MANEMP_QoQ',
       'ICSA_QoQ', 'IC4WSA_QoQ', 'RRSFS_QoQ', '^GSPC_QoQ', '^VIX_QoQ',
       'DX-Y.NYB_QoQ', 'M2SL_QoQ', 'TB6SMFFM', 'T10Y2Y', 'T2Y3M', 'T5Y2Y',
       'prev1_decision', 'prev2_decision'],
      dtype='object')


,CPIAUCSL_QoQ,CPILFESL_QoQ,PPIACO_QoQ,CL=F_QoQ,GDP_QoQ,GDPC1_QoQ,INDPRO_QoQ,UNRATE_QoQ,PAYEMS_QoQ,MANEMP_QoQ,...,^GSPC_QoQ,^VIX_QoQ,DX-Y.NYB_QoQ,M2SL_QoQ,TB6SMFFM,T10Y2Y,T2Y3M,T5Y2Y,prev1_decision,prev2_decision
2024-06-30,0.328093,0.499113,0.321057,-1.959838,1.371730,0.738980,0.716748,5.128205,0.252671,0.007789,...,3.923033,-4.381250,1.262553,0.785593,-0.18,-0.35,-0.77,-0.42,5.33,5.33
2024-09-30,0.549291,0.782803,-1.262924,-16.396864,1.234461,0.759510,-0.637267,0.000000,0.252668,-0.303762,...,5.530649,34.485531,-4.807787,0.980271,-0.71,0.15,-1.30,-0.12,5.33,5.33
2024-12-31,0.874064,0.767061,0.297607,5.207574,1.187918,0.607065,0.509087,0.000000,0.396680,-0.312500,...,2.067684,3.705923,7.650327,1.023876,-0.29,0.33,-0.16,0.02,4.64,4.83
2025-03-31,0.633495,0.730909,2.131916,-0.334632,0.000000,0.000000,0.748174,2.439024,0.286897,0.031348,...,-4.586820,28.414987,-3.945063,1.268503,-0.23,0.34,-0.37,0.07,4.33,4.33
2025-06-30,0.000000,0.000000,0.000000,-12.856746,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-4.205213,27.692998,-4.236638,0.000000,NaN,0.59,NaN,NaN,NaN,NaN


## Group 5 Michigan Survey Data

In [19]:
# michigan survey data: load second sheet and skip metadata
# unemployent
unemp_df = pd.read_excel("ylch26h.xls", sheet_name=1, skiprows=6)
# business 
biz_df = pd.read_excel("ylch30h.xls", sheet_name=1, skiprows=6)
# interest rates
ir_df = pd.read_excel("ylch31h.xls", sheet_name=1, skiprows=6)

unemp_df = unemp_df.rename(columns={unemp_df.columns[0]: "Date", unemp_df.columns[2]: "unemp_idx"})
biz_df = biz_df.rename(columns={biz_df.columns[0]: "Date", biz_df.columns[2]: "b_idx"})
ir_df = ir_df.rename(columns={ir_df.columns[0]: "Date", ir_df.columns[2]: "ir_idx"})

for df in [unemp_df, biz_df, ir_df]:
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df.dropna(subset=['Date'], inplace=True)

merged_survey = unemp_df[['Date', 'unemp_idx']] \
    .merge(biz_df[['Date', 'b_idx']], on='Date') \
    .merge(ir_df[['Date', 'ir_idx']], on='Date')

merged_survey.set_index('Date', inplace=True)

merged_survey = merged_survey[merged_survey.index >= '1994-01-01']

survey_q_level = merged_survey.resample('Q').last()

print(survey_q_level.head())

data_final_merged = data_final.merge(survey_q_level, left_index=True, right_index=True, how='left')

            unemp_idx  b_idx  ir_idx
Date                                
1994-03-31      117.0   90.0    46.0
1994-06-30      109.0   85.0    32.0
1994-09-30      107.0   84.0    30.0
1994-12-31      112.0   88.0    27.0
1995-03-31      112.0   83.0    34.0


In [20]:
data_final_merged.head()

,CPIAUCSL_QoQ,CPILFESL_QoQ,PPIACO_QoQ,CL=F_QoQ,GDP_QoQ,GDPC1_QoQ,INDPRO_QoQ,UNRATE_QoQ,PAYEMS_QoQ,MANEMP_QoQ,...,M2SL_QoQ,TB6SMFFM,T10Y2Y,T2Y3M,T5Y2Y,prev1_decision,prev2_decision,unemp_idx,b_idx,ir_idx
1994-03-31,0.546822,0.648088,0.927487,NaN,1.453063,0.970363,1.378298,0.000000,0.819949,0.487660,...,0.161174,0.44,1.56,1.41,0.94,3.25,3.05,117.0,90.0,46.0
1994-06-30,0.543848,0.708307,0.668338,NaN,1.844933,1.355183,1.740912,-6.153846,0.877743,0.662840,...,-0.017241,0.30,1.15,1.68,0.77,4.01,3.56,109.0,85.0,32.0
1994-09-30,0.946586,0.703325,0.414938,NaN,1.161098,0.584585,1.090393,-3.278689,0.882361,0.617320,...,0.189682,0.31,1.00,1.64,0.69,4.47,4.26,107.0,84.0,30.0
1994-12-31,0.535834,0.507937,0.743802,NaN,1.694335,1.145570,2.498505,-6.779661,0.803492,0.607690,...,0.008606,0.76,0.15,1.83,0.19,5.29,4.76,112.0,88.0,27.0
1995-03-31,0.732845,1.010739,1.640689,NaN,0.898704,0.354781,0.214123,-1.818182,0.632677,0.261354,...,0.134810,-0.09,0.40,0.87,0.27,5.92,5.53,112.0,83.0,34.0


## Group 5 PMI from refintive

In [21]:
pmi_df = pd.read_csv("pmi_quarterly.csv", parse_dates=['Date'], index_col='Date')

# Step 2: Merge PMI with your existing DataFrame
data_final_merged = data_final_merged.merge(pmi_df[['pmi_QoQ']], left_index=True, right_index=True, how='left')

In [22]:
print(data_final_merged.tail())

            CPIAUCSL_QoQ  CPILFESL_QoQ  PPIACO_QoQ   CL=F_QoQ   GDP_QoQ  \
2024-06-30      0.328093      0.499113    0.321057  -1.959838  1.371730   
2024-09-30      0.549291      0.782803   -1.262924 -16.396864  1.234461   
2024-12-31      0.874064      0.767061    0.297607   5.207574  1.187918   
2025-03-31      0.633495      0.730909    2.131916  -0.334632  0.000000   
2025-06-30      0.000000      0.000000    0.000000 -12.856746  0.000000   

            GDPC1_QoQ  INDPRO_QoQ  UNRATE_QoQ  PAYEMS_QoQ  MANEMP_QoQ  ...  \
2024-06-30   0.738980    0.716748    5.128205    0.252671    0.007789  ...   
2024-09-30   0.759510   -0.637267    0.000000    0.252668   -0.303762  ...   
2024-12-31   0.607065    0.509087    0.000000    0.396680   -0.312500  ...   
2025-03-31   0.000000    0.748174    2.439024    0.286897    0.031348  ...   
2025-06-30   0.000000    0.000000    0.000000    0.000000    0.000000  ...   

            TB6SMFFM  T10Y2Y  T2Y3M  T5Y2Y  prev1_decision  prev2_decision  \
20

In [23]:
data_final_merged.shape

(126, 27)

In [26]:
nan_counts = data_final_merged.isna().sum(axis=1)

# Get rows with most NaN values (sorted descending)
rows_with_most_nans = nan_counts.sort_values(ascending=False)

# Display top 5 rows with most NaN values
print("Rows with most NaN values:")
print(rows_with_most_nans.head())

# Optionally display the actual data for these rows
print("\nData for these rows:")
print(data_final_merged.loc[rows_with_most_nans.head().index])

Rows with most NaN values:
2025-06-30    9
1994-03-31    4
1995-06-30    1
1994-12-31    1
2000-09-30    1
dtype: int64

Data for these rows:
            CPIAUCSL_QoQ  CPILFESL_QoQ  PPIACO_QoQ   CL=F_QoQ   GDP_QoQ  \
2025-06-30      0.000000      0.000000    0.000000 -12.856746  0.000000   
1994-03-31      0.546822      0.648088    0.927487        NaN  1.453063   
1995-06-30      0.793651      0.750469    1.129944        NaN  0.780454   
1994-12-31      0.535834      0.507937    0.743802        NaN  1.694335   
2000-09-30      0.813008      0.662617    0.672646        NaN  0.687421   

            GDPC1_QoQ  INDPRO_QoQ  UNRATE_QoQ  PAYEMS_QoQ  MANEMP_QoQ  ...  \
2025-06-30   0.000000    0.000000    0.000000    0.000000    0.000000  ...   
1994-03-31   0.970363    1.378298    0.000000    0.819949    0.487660  ...   
1995-06-30   0.298323    0.635666    3.703704    0.327608   -0.092684  ...   
1994-12-31   1.145570    2.498505   -6.779661    0.803492    0.607690  ...   
2000-09-30   0.10

In [29]:
# 27 variables in paper 
assert data_final_merged.shape[1] == 27

In [32]:
# finalize and export
df_to_export = data_final_merged.reset_index()
df_to_export.to_csv('predictor_df.csv', index=False)